In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
data = pd.read_csv('E:\\Programing\\Python\\The-Complete-Neural-Networks-Bootcamp-Theory-Applications-master\\The-Complete-Neural-Networks-Bootcamp-Theory-Applications-master\\diabetes.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


> For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). 
    
> For y: Extract out the last column (which is the label).

> Convert both to numpy using the .values method.

In [3]:
x = data.iloc[:, 0:-1].values
y_string = list(data.iloc[:,-1])

    Our neural network only understand numbers! So convert the string to labels


In [4]:
y_int = []

for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

    Now convert to an array


In [5]:
y = np.array(y_int, dtype='float64')

    Feature Normalization. All features should have the same range of values (-1,1)

In [6]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [7]:
# Now we convert the arrays to PyTorch tensors
x = torch.tensor(x)

# We add an extra dimension to convert this array to 2D
y = torch.tensor(y).unsqueeze(1)

In [8]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [9]:
class Dataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__ (self):
        return len(self.x)

In [10]:
dataset = Dataset(x, y)
len(dataset)

768

    Load the data to your dataloader for batch processing and shuffling


In [11]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True)

In [12]:
print("There is {} barches in the dataset".format(len(train_loader)))

for (x,y) in train_loader:
    print("For one iteration (batch), there is: ")
    print("Data:    {}".format(x.shape))
    print("Labels:      {}".format(y.shape))
    break

There is 24 barches in the dataset
For one iteration (batch), there is: 
Data:    torch.Size([32, 7])
Labels:      torch.Size([32, 1])


In [13]:
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5,4)
        self.fc3 = nn.Linear(4,3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)

        out = self.fc2(out)
        out = self.tanh(out)

        out = self.fc3(out)
        out = self.tanh(out)

        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [14]:
# Create the network (an object of the Net class)
net = Model(7, 1)

#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True)

# We will use SGD with momentum with a learning rate of 0.1
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)

c:\Users\sviat\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


    Train the network 

In [16]:
epochs = 350
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()

        outputs = net(inputs)
        
        loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    output = (outputs > 0.5).float()
    accuracy = (output == labels).float().mean()

    print("Epoch {}/{}, loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1, epochs, loss, accuracy))

Epoch 1/350, loss: 0.457, Accuracy: 0.781
Epoch 2/350, loss: 0.241, Accuracy: 0.906
Epoch 3/350, loss: 0.268, Accuracy: 0.938
Epoch 4/350, loss: 0.679, Accuracy: 0.719
Epoch 5/350, loss: 0.570, Accuracy: 0.719
Epoch 6/350, loss: 0.564, Accuracy: 0.719
Epoch 7/350, loss: 0.377, Accuracy: 0.844
Epoch 8/350, loss: 0.407, Accuracy: 0.812
Epoch 9/350, loss: 0.497, Accuracy: 0.812
Epoch 10/350, loss: 0.485, Accuracy: 0.781
Epoch 11/350, loss: 0.377, Accuracy: 0.781
Epoch 12/350, loss: 0.469, Accuracy: 0.781
Epoch 13/350, loss: 0.593, Accuracy: 0.750
Epoch 14/350, loss: 0.339, Accuracy: 0.812
Epoch 15/350, loss: 0.202, Accuracy: 0.969
Epoch 16/350, loss: 0.388, Accuracy: 0.781
Epoch 17/350, loss: 0.327, Accuracy: 0.844
Epoch 18/350, loss: 0.536, Accuracy: 0.656
Epoch 19/350, loss: 0.524, Accuracy: 0.719
Epoch 20/350, loss: 0.433, Accuracy: 0.781
Epoch 21/350, loss: 0.404, Accuracy: 0.812
Epoch 22/350, loss: 0.423, Accuracy: 0.781
Epoch 23/350, loss: 0.262, Accuracy: 0.906
Epoch 24/350, loss: 